In [ ]:
# numpy
import numpy as np
# matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# 구글드라이브를 /content/gdrive/'My Drive'에 마운트

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# 마운트된 구글 드라이브로 이동

%cd /content/gdrive/MyDrive/ssu/dl/prj

/content/gdrive/MyDrive/ssu/dl/prj


In [ ]:
!git clone https://github.com/ssuai/deep_learning_from_scratch_2.git

fatal: destination path 'deep_learning_from_scratch_2' already exists and is not an empty directory.


In [ ]:
%cd deep_learning_from_scratch_2/

/content/gdrive/MyDrive/ssu/dl/prj/deep_learning_from_scratch_2


In [ ]:
from common.time_layers import *
from ch08.attention_layer import TimeAttention
from ch01.forward_net import Affine


class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout

        return dx


class Encoder:
    def __init__(self, vocab_size, wordvec_size, hidden_size):
        V, D, H = vocab_size, wordvec_size, hidden_size
        rn = np.random.randn

        embed_W = (rn(V, D) / 100).astype('f')
        # lstm_Wx = (rn(D, 4 * H) / np.sqrt(D)).astype('f')
        # lstm_Wh = (rn(H, 4 * H) / np.sqrt(H)).astype('f')
        w = (rn(D, H) / np.sqrt(D)).astype('f')
        b = np.zeros(H).astype('f')
        # lstm_b = np.zeros(4 * H).astype('f')

        self.embed = TimeEmbedding(embed_W)
        self.attention = TimeAttention()
        self.affine = TimeAffine(w, b)
        self.relu = Relu()

        self.params = self.embed.params + self.affine.params
        self.grads = self.embed.grads + self.affine.grads
        self.hs = None

    def forward(self, xs):
        xs = self.embed.forward(xs)
        xs = self.attention.forward(xs, xs)
        xs = self.affine.forward(xs)
        hs = self.relu.forward(xs)

        self.hs = hs
        return hs[:, -1, :]

    def backward(self, dh):
        dout = self.relu.backward(dh)
        dout = self.affine.backward(dout)
        dout = self.attention.backward(dout)
        dout = self.embed.backward(dout)
        return dout